In [0]:
pip install tika

  Created wheel for tika: filename=tika-1.23.1-cp36-none-any.whl size=32561 sha256=763574a8d7c5e5e3a592eec6c8af54bb40034fcf8ae363207f32986799aba08e
  Stored in directory: /root/.cache/pip/wheels/fd/6b/6d/c850c2a934057edce9779d41400d910c6a9b1f22027566b10f
Successfully built tika


In [0]:
mkdir USE/

In [0]:
!curl -L 'https://tfhub.dev/google/universal-sentence-encoder-large/3?tf-hub-format=compressed' | tar -zxvC USE/

  % Total    % Received % Xferd  Average Speed   Time    Time     Time  Current
                                 Dload  Upload   Total   Spent    Left  Speed
  0     0    0     0    0     0      0      0 --:--:-- --:--:-- --:--:--     0
  0  745M    0  130k    0     0   688k      0  0:18:29 --:--:--  0:18:29  688k./
./tfhub_module.pb
./variables/
./variables/variables.data-00000-of-00001
 95  745M   95  712M    0     0  43.9M      0  0:00:16  0:00:16 --:--:-- 40.1M./variables/variables.index
./assets/
./saved_model.pb
100  745M  100  745M    0     0  44.1M      0  0:00:16  0:00:16 --:--:-- 42.1M


In [0]:
import tensorflow as tf
import tensorflow_hub as hub
def embed_useT(module):
    with tf.Graph().as_default():
        sentences = tf.placeholder(tf.string)
        embed = hub.Module(module)
        embeddings = embed(sentences)
        session = tf.train.MonitoredSession()
    return lambda x: session.run(embeddings, {sentences: x})
embed_fn = embed_useT('USE/')


INFO:tensorflow:Saver not created because there are no variables in the graph to restore
INFO:tensorflow:Graph was finalized.
INFO:tensorflow:Running local_init_op.
INFO:tensorflow:Done running local_init_op.


In [0]:
import tensorflow as tf
import tensorflow_hub as hub
import numpy as np

greets = ["What's up?",
 'It is a pleasure to meet you.',
 'How do you do?',
 'Top of the morning to you!',
 'Hi',
 'How are you doing?',
 'Hello',
 'Greetings!',
 'Hi, How is it going?',
 'Hi, nice to meet you.',
 'Nice to meet you.']
greet_matrix = embed_fn(greets)
test_text = "Hey, how are you?"
test_embed = embed_fn([test_text])
np.inner(test_embed, greet_matrix)
sim_matrix  = np.inner(test_embed, greet_matrix)
if sim_matrix.max() > 0.8:
    print("it is a greetings")
else:
    print("it is not a greetings")

it is a greetings


In [0]:
mkdir data

In [0]:
from tika import parser
import glob
import re
import tensorflow as tf
import tensorflow_hub as hub
import numpy as np


# question = ["What model was used in this paper and what was the model's performance"]
question = ['What is the architecture of the models used?']
question_matrix = embed_fn(question)
dir = 'data/*.pdf'
# print(glob.glob('*/'))
# print(glob.glob(dir))
for research_paper_name in glob.glob(dir):
  print(research_paper_name)
  research_paper_data = parser.from_file(research_paper_name)
  research_paper_content = research_paper_data['content']
  paragraphs = research_paper_content.split(". ")
  # print(paragraphs)
  research_paper_txt_name = research_paper_name.replace("pdf","txt")
  rp_file= open(research_paper_txt_name,"w+")
  data_file= open('data/data.txt',"a+")
  edited_paragraphs = []
  for i in range(len(paragraphs)):
    rp_file.write(str(paragraphs[i]))
    if paragraphs[i] != ' ':
      edited_paragraphs.append(paragraphs[i])
    if len(edited_paragraphs) != 0:
      edited_paragraphs[len(edited_paragraphs)-1] = re.sub(r"\n", "", edited_paragraphs[len(edited_paragraphs)-1] )
  data_file.write(research_paper_txt_name + '\n\n')
  data_file.write('------------------------------- \n')
  for paragraph in edited_paragraphs:
    test_text = paragraph
    test_embed = embed_fn([test_text])
    np.inner(test_embed, question_matrix)
    sim_matrix  = np.inner(test_embed, question_matrix)
    if sim_matrix.max() > 0.3:
        # print(paragraph)
        data_file.write(paragraph + '\n\n')
    else:
      pass
  data_file.write(''' 
  

  
  '''
  )
  rp_file.close()
data_file.close()
print("Done!")
        
  
  # print(edited_paragraphs)
  
  

data/sharing_knowledge_in_multi_task_deep_reinforcement_learning.pdf
data/sqil_imitation_learning_via_reinforcement_learning_with_sparse_rewards.pdf
data/structpool_structured_graph_pooling_via_conditional_random_fields.pdf
data/reinforcement_learning_based_graph_to_sequence_model_for_natural_question_generation.pdf
data/self_learning_to_filter_noisy_labels_with_self_ensembling.pdf
Done!
